[Reference](https://python.plainenglish.io/one-page-reports-with-python-adf58432ae46)

In [1]:
# Importing the necessary modules to the email sending.
import smtplib
from email.message import EmailMessage
my_email = 'Write your e-mail address here'
my_password = 'Write your e-mail password here'

# Through this nested loop, we are going to obtain the stores' general informations
# (manager name and e-mail) and their daily performance data.
for store_id in stores['Store ID']:
  
    # Getting the store's basic informations.
    store_name = stores[stores['Store ID'] == store_id]['Store'].values[0]
    manager = emails[emails['Store'] == store_name]['Manager'].values[0]
    manager_email = emails[emails['Store'] == store_name]['E-mail'].values[0]
    
    # For each date when a given store has made a selling, the day's
    # total revenue along the number of different products sold and the average ticket
    # will be stored in variables.
    for date in sellings[sellings['Store ID'] == store_id]['Date'].unique():
        revenue = revenues.xs([date, store_id], level=[0,1]).values[0]
        no_different_products = (different_products.xs([date, store_id], level=[0,1])
                                                                     .values[0])
        ticket = average_ticket.xs([date, store_id], level=[0,1]).values[0]
        
        # Creating the email message. The report will be displayed as an HTML table.
        msg = EmailMessage()
        msg['From'] = 'Write_your_email@gmail.com'
        msg['To'] = manager_email
        msg['Subject'] = f'{pd.to_datetime(date).date()} One Page Report'
        
        # 'add_alternative' enables us to use HTML structure in the message. Just remember 
        # to set the 'subtype' argument to 'html'.
        msg.add_alternative(
            f'''
            <p>Dear {manager},</p>
            <p>You can see below the One Page Report produced in respect to your store\'s sales
            performance on {pd.to_datetime(date).date()}.</p>
            <br>
            <table>
                <tr>
                    <th>
                        Metric
                    </th>
                    <th>
                        Expected Value
                    </th>
                    <th>
                         Value Obtained
                    </th>
                    <th>
                         Metric Status
                    </th>                    
                </tr>
                
                <tr>
                    <td>
                        Revenue
                    </td>
                    <td>
                        R$ 1,000.00
                    </td>
                    <td>
                        R$ {revenue :,.2f}
                    </td>
                    <td>
                        <font color={'green' if revenue >=1000 else 'red'}>
                            {'Achieved' if revenue >=1000 else 'Failed'} </font>
                    </td>
                    
                </tr>
                
                <tr>
                    <td>
                        No. of Diff. Products
                    </td>
                    <td>
                        4
                    </td>
                    <td>
                        {no_different_products}
                    </td>
                    <td>
                        <font color={'green' if no_different_products >=4 else 'red'}>
                            {'Achieved' if no_different_products >=4 else 'Failed'} </font>
                    </td>
                </tr>
                
                <tr>
                    <td>
                        Average Ticket
                    </td>
                    <td>
                        R$ 500.00
                    </td>
                    <td>
                        R$ {ticket :,.2f}
                    </td>
                    <td>
                        <font color={'green' if ticket >=500 else 'red'}>
                            {'Achieved' if ticket >=500 else 'Failed'} </font>
                    </td> 
                </tr>
            </table>
            <br>
            <p>Best Regards,</p>
            <p>The Data Analysis Team</p>
            '''
            ,subtype='html')
        
        # Logging into our email account to send the emails.
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(my_email, my_password)
            smtp.send_message(msg)
        